In [13]:
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [15]:
import lightgbm as lgb 
import xgboost as xgb 
import catboost as cbt 
import numpy as np 
import joblib 
import os
import sklearn 

In [16]:
indices = ['stock_id', 'date_id','time_id', 'row_id','target']

features = ['seconds_in_bucket', 'imbalance_buy_sell_flag',
            'imbalance_size', 'matched_size', 'bid_size', 'ask_size',
            'reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap',
            'imb_s1', 'imb_s2']

In [17]:
def enrich_features(features):
   
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            if i>j:
                features.append(f'{a}_{b}_imb')    
                    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            for k,c in enumerate(prices):
                if i>j and j>k:
                    features.append(f'{a}_{b}_{c}_imb2')
                    
    features = set(features)


In [18]:
enrich_features(features)

In [19]:
features

['seconds_in_bucket',
 'imbalance_buy_sell_flag',
 'imbalance_size',
 'matched_size',
 'bid_size',
 'ask_size',
 'reference_price',
 'far_price',
 'near_price',
 'ask_price',
 'bid_price',
 'wap',
 'imb_s1',
 'imb_s2',
 'far_price_reference_price_imb',
 'near_price_reference_price_imb',
 'near_price_far_price_imb',
 'ask_price_reference_price_imb',
 'ask_price_far_price_imb',
 'ask_price_near_price_imb',
 'bid_price_reference_price_imb',
 'bid_price_far_price_imb',
 'bid_price_near_price_imb',
 'bid_price_ask_price_imb',
 'wap_reference_price_imb',
 'wap_far_price_imb',
 'wap_near_price_imb',
 'wap_ask_price_imb',
 'wap_bid_price_imb',
 'near_price_far_price_reference_price_imb2',
 'ask_price_far_price_reference_price_imb2',
 'ask_price_near_price_reference_price_imb2',
 'ask_price_near_price_far_price_imb2',
 'bid_price_far_price_reference_price_imb2',
 'bid_price_near_price_reference_price_imb2',
 'bid_price_near_price_far_price_imb2',
 'bid_price_ask_price_reference_price_imb2',
 'b

In [20]:
def enrich_df(df):

    df_ = df.copy()

    df_['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df_['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            if i>j:
                df_[f'{a}_{b}_imb'] = df_.eval(f'({a}-{b})/({a}+{b})')
#                 features.append(f'{a}_{b}_imb')    
                    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            for k,c in enumerate(prices):
                if i>j and j>k:
                    max_ = df_[[a,b,c]].max(axis=1)
                    min_ = df_[[a,b,c]].min(axis=1)
                    mid_ = df_[[a,b,c]].sum(axis=1)-min_-max_

                    df_[f'{a}_{b}_{c}_imb2'] = (max_-mid_)/(mid_-min_ + 0.1)
#                     features.append(f'{a}_{b}_{c}_imb2')
    
    return df_

# TRAINING = True
# if TRAINING:
#     df_train = pd.read_csv(f'{DATA_PATH}/train.csv')
#     df_ = generate_features(df_train)


In [21]:
model_dict = {
    'lgb': lgb.LGBMRegressor(objective='regression_l1', n_estimators=50),
    'xgb': xgb.XGBRegressor(tree_method='hist', objective='reg:absoluteerror', n_estimators=500),
    'cbt': cbt.CatBoostRegressor(objective='MAE', iterations=50),
}

In [22]:
models = []

Train


In [23]:
# DATA_PATH = '/kaggle/input'
DATA_PATH = '..'

In [24]:
df_train = pd.read_csv(f'{DATA_PATH}/optiver-trading-at-the-close/train.csv')
df_test = pd.read_csv(f'{DATA_PATH}/optiver-trading-at-the-close/example_test_files/test.csv')
revealed_targets = pd.read_csv(f'{DATA_PATH}/optiver-trading-at-the-close/example_test_files/revealed_targets.csv')
sample_submission = pd.read_csv(f'{DATA_PATH}/optiver-trading-at-the-close/example_test_files/sample_submission.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/optiver-trading-at-the-close/train.csv'

In [ ]:
df_train.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id'],
      dtype='object')

In [ ]:
df_train_ = enrich_df(df_train)

In [ ]:
df_train_.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id', 'imb_s1', 'imb_s2',
       'far_price_reference_price_imb', 'near_price_reference_price_imb',
       'near_price_far_price_imb', 'ask_price_reference_price_imb',
       'ask_price_far_price_imb', 'ask_price_near_price_imb',
       'bid_price_reference_price_imb', 'bid_price_far_price_imb',
       'bid_price_near_price_imb', 'bid_price_ask_price_imb',
       'wap_reference_price_imb', 'wap_far_price_imb', 'wap_near_price_imb',
       'wap_ask_price_imb', 'wap_bid_price_imb',
       'near_price_far_price_reference_price_imb2',
       'ask_price_far_price_reference_price_imb2',
       'ask_price_near_price_reference_price_imb2',
       'ask_price_near_price_far_price_imb2',
       'bid_price_far_price_reference_price_imb2',
  

In [ ]:
os.system('mkdir models')

mkdir: cannot create directory ‘models’: File exists


256

In [ ]:
df_train_.isna().sum()

stock_id                                           0
date_id                                            0
seconds_in_bucket                                  0
imbalance_size                                   220
imbalance_buy_sell_flag                            0
reference_price                                  220
matched_size                                     220
far_price                                    2894342
near_price                                   2857180
bid_price                                        220
bid_size                                           0
ask_price                                        220
ask_size                                           0
wap                                              220
target                                            88
time_id                                            0
row_id                                             0
imb_s1                                            87
imb_s2                                        

In [ ]:
df_train_[features].columns

Index(['seconds_in_bucket', 'imbalance_buy_sell_flag', 'imbalance_size',
       'matched_size', 'bid_size', 'ask_size', 'reference_price', 'far_price',
       'near_price', 'ask_price', 'bid_price', 'wap', 'imb_s1', 'imb_s2',
       'far_price_reference_price_imb', 'near_price_reference_price_imb',
       'near_price_far_price_imb', 'ask_price_reference_price_imb',
       'ask_price_far_price_imb', 'ask_price_near_price_imb',
       'bid_price_reference_price_imb', 'bid_price_far_price_imb',
       'bid_price_near_price_imb', 'bid_price_ask_price_imb',
       'wap_reference_price_imb', 'wap_far_price_imb', 'wap_near_price_imb',
       'wap_ask_price_imb', 'wap_bid_price_imb',
       'near_price_far_price_reference_price_imb2',
       'ask_price_far_price_reference_price_imb2',
       'ask_price_near_price_reference_price_imb2',
       'ask_price_near_price_far_price_imb2',
       'bid_price_far_price_reference_price_imb2',
       'bid_price_near_price_reference_price_imb2',
       'bid

In [ ]:
df_train_.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id', 'imb_s1', 'imb_s2',
       'far_price_reference_price_imb', 'near_price_reference_price_imb',
       'near_price_far_price_imb', 'ask_price_reference_price_imb',
       'ask_price_far_price_imb', 'ask_price_near_price_imb',
       'bid_price_reference_price_imb', 'bid_price_far_price_imb',
       'bid_price_near_price_imb', 'bid_price_ask_price_imb',
       'wap_reference_price_imb', 'wap_far_price_imb', 'wap_near_price_imb',
       'wap_ask_price_imb', 'wap_bid_price_imb',
       'near_price_far_price_reference_price_imb2',
       'ask_price_far_price_reference_price_imb2',
       'ask_price_near_price_reference_price_imb2',
       'ask_price_near_price_far_price_imb2',
       'bid_price_far_price_reference_price_imb2',
  

In [ ]:
df_train_.shape[0]

5237980

In [ ]:
X = df_train_.dropna(subset=['target'])[features].values
X.shape

(5237892, 49)

In [ ]:
Y = df_train_.dropna(subset=['target'])['target'].values
Y.shape

(5237892,)

In [ ]:
N_fold = 5

Train

In [ ]:
index = np.arange(len(X))

In [ ]:
def train(model_dict, modelname, fold):
    model = model_dict[modelname]
    model.fit(X[index%N_fold!=fold], Y[index%N_fold!=fold], 
                eval_set=[(X[index%N_fold==fold], Y[index%N_fold==fold])], 
                callbacks=[lgb.early_stopping(100)]
                )
    # model.early_stopping(100)
    models.append(model)
    joblib.dump(model, f'./models/{modelname}_{fold}.model')

In [ ]:
for i in range(N_fold):
    print(f"Training fold {i}")
    train(model_dict, 'lgb', i)
#     train(model_dict, 'xgb', i)
    train(model_dict, 'cbt', i)

Training fold 0
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[50]	valid_0's l1: 6.37513
0:	learn: 6.3764794	test: 6.5089182	best: 6.5089182 (0)	total: 1.49s	remaining: 1m 13s


SystemError: <method '_train' of '_catboost._CatBoost' objects> returned a result with an exception set

Load

In [ ]:
# def load(model_dict, modelname, fold):
#     models.append(joblib.load(f'models/{modelname}_{fold}.model'))

In [ ]:
# for i in range (0, 5):
#     load(model_dict, 'lgb', i)

In [ ]:
models

Submit

In [ ]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [ ]:
len(features)

In [ ]:
# iter_test = env.iter_test()

In [ ]:
counter = 0
# sample_prediction['target'] = 0
# env.predict(sample_prediction)
for (test, revealed_targets, sample_prediction) in iter_test:
#     print(test.shape)
    test_ = enrich_df(test)[features]
#     print(test_.shape)
#     print(len(features))
#     print(len(test_))
    sample_prediction['target'] = np.mean([model.predict(test_) for model in models], 0)
    env.predict(sample_prediction)
    counter += 1

In [ ]:
a = 1
a